In [1]:
! git clone https://github.com/amazon-science/mintaka

Cloning into 'mintaka'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 39 (delta 16), reused 25 (delta 8), pack-reused 0
Unpacking objects: 100% (39/39), 7.53 MiB | 2.66 MiB/s, done.


In [1]:
import json
import pandas as pd

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
#dev

with open('mintaka/data/mintaka_dev.json') as user_file:
    dev = user_file.read()

parsed_dev = json.loads(dev)

dev = pd.DataFrame(columns=['question', 'answer_type'])

for i in range(len(parsed_dev)):
    dev.loc[i, 'question'] = parsed_dev[i]['question']
    dev.loc[i, 'answer_type'] = parsed_dev[i]['answer']['answerType']

In [4]:
#test

with open('mintaka/data/mintaka_test.json') as user_file:
    test = user_file.read()

parsed_test = json.loads(test)

test = pd.DataFrame(columns=['question', 'answer_type'])

for i in range(len(parsed_test)):
    test.loc[i, 'question'] = parsed_test[i]['question']
    test.loc[i, 'answer_type'] = parsed_test[i]['answer']['answerType']

In [5]:
#train

with open('mintaka/data/mintaka_train.json') as user_file:
    train = user_file.read()

parsed_train = json.loads(train)

train = pd.DataFrame(columns=['question', 'answer_type'])

for i in range(len(parsed_train)):
    train.loc[i, 'question'] = parsed_train[i]['question']
    train.loc[i, 'answer_type'] = parsed_train[i]['answer']['answerType']

In [6]:
train.head()

,question,answer_type
0,What is the seventh tallest mountain in North ...,entity
1,Which actor was the star of Titanic and was bo...,entity
2,Which actor starred in Vanilla Sky and was mar...,entity
3,What year was the first book of the A Song of ...,date
4,Who is the youngest current US governor?,entity


In [6]:
import numpy as np
train['answer_gen_type'] = np.nan
test['answer_gen_type'] = np.nan
dev['answer_gen_type'] = np.nan

In [7]:
for i in range(len(train)):
    if train.loc[i, 'answer_type'] == "entity":
        train.loc[i, 'answer_gen_type'] = "entity"
    else:
        train.loc[i, 'answer_gen_type'] = "other"

In [8]:
for i in range(len(test)):
    if test.loc[i, 'answer_type'] == "entity":
        test.loc[i, 'answer_gen_type'] = "entity"
    else:
        test.loc[i, 'answer_gen_type'] = "other"

In [9]:
for i in range(len(dev)):
    if dev.loc[i, 'answer_type'] == "entity":
        dev.loc[i, 'answer_gen_type'] = "entity"
    else:
        dev.loc[i, 'answer_gen_type'] = "other"

In [10]:
train.answer_gen_type.value_counts()

entity    8777
other     5223
Name: answer_gen_type, dtype: int64

In [11]:
test.answer_gen_type.value_counts()

entity    2499
other     1501
Name: answer_gen_type, dtype: int64

In [12]:
dev.answer_gen_type.value_counts()

entity    1248
other      752
Name: answer_gen_type, dtype: int64

In [3]:
! pip install simpletransformers


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [10]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

In [11]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [12]:
train_data = []
for i in range(len(train)):
    if train.loc[i, 'answer_gen_type'] == 'entity':
        cur_train = [train.loc[i, 'question'], 1]
    else:
        cur_train = [train.loc[i, 'question'], 0]
    train_data.append(cur_train)

In [13]:
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]

In [14]:
eval_data = []
for i in range(len(dev)):
    if dev.loc[i, 'answer_gen_type'] == 'entity':
        cur_eval = [dev.loc[i, 'question'], 1]
    else:
        cur_eval = [dev.loc[i, 'question'], 0]
    eval_data.append(cur_eval)

eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text", "labels"]

In [18]:
model_args = ClassificationArgs(num_train_epochs=3,
                                use_multiprocessing=False,
                                use_multiprocessing_for_evaluation=False)

In [19]:
model = ClassificationModel(
    "bert", "bert-base-uncased", args=model_args
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [20]:
model.train_model(train_df)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_2_2


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/1750 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/1750 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/1750 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.


(5250, 0.08880995978060223)

In [21]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2


Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.9595611343256295, 'tp': 1227, 'tn': 735, 'fp': 17, 'fn': 21, 'auroc': 0.9978641358087835, 'auprc': 0.9988359026209508, 'eval_loss': 0.09227217745780945}


In [22]:
result

{'mcc': 0.9595611343256295,
 'tp': 1227,
 'tn': 735,
 'fp': 17,
 'fn': 21,
 'auroc': 0.9978641358087835,
 'auprc': 0.9988359026209508,
 'eval_loss': 0.09227217745780945}

In [23]:
eval_data2 = []
for i in range(len(test)):
    if test.loc[i, 'answer_gen_type'] == 'entity':
        cur_eval = [test.loc[i, 'question'], 1]
    else:
        cur_eval = [test.loc[i, 'question'], 0]
    eval_data2.append(cur_eval)

eval_df2 = pd.DataFrame(eval_data2)
eval_df2.columns = ["text", "labels"]

In [24]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df2)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2


Running Evaluation:   0%|          | 0/500 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.9653063270974935, 'tp': 2477, 'tn': 1458, 'fp': 43, 'fn': 22, 'auroc': 0.9975617162254642, 'auprc': 0.998516956182094, 'eval_loss': 0.08744728249311447}


In [25]:
result

{'mcc': 0.9653063270974935,
 'tp': 2477,
 'tn': 1458,
 'fp': 43,
 'fn': 22,
 'auroc': 0.9975617162254642,
 'auprc': 0.998516956182094,
 'eval_loss': 0.08744728249311447}

In [28]:
test.answer_gen_type.value_counts()

entity    2499
other     1501
Name: answer_gen_type, dtype: int64

In [31]:
test.loc[1]['question']

'How many Academy Awards has Jake Gyllenhaal been nominated for?'